In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

In [2]:
df = pd.read_csv('data_treated.csv')
df

,Run,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,...,xNH3_cal,xN2_cal,xH2_cal,Fin [mmol/min],Fout [mmol/min],Vin [mL/min],Vout [mL/min],Vavg [mL/min],Vcat [mL],rt [min]
0,0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,...,0.925,0.01875,0.05625,1.226182,1.273955,57.670636,59.917543,58.786933,0.402439,0.006062
1,1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,...,0.948,0.01300,0.03900,2.452364,2.517827,115.341271,118.420196,116.873975,0.402439,0.003049
2,2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,...,0.951,0.01225,0.03675,3.678546,3.770934,173.011907,177.357157,175.175550,0.402439,0.002034
3,3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,...,0.795,0.05125,0.15375,1.226182,1.366219,62.701660,69.862574,66.217597,0.402439,0.005382
4,4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,...,0.843,0.03925,0.11775,2.452364,2.661274,125.403320,136.086078,130.671929,0.402439,0.002727
5,5,0.3,350,90,3.678546,0.875,0.031,0.094,0.066667,3.433310,...,0.875,0.03125,0.09375,3.678546,3.923782,188.104981,200.645313,194.307707,0.402439,0.001834
6,6,0.3,400,30,1.226182,0.514,0.122,0.365,0.321004,0.832573,...,0.514,0.12150,0.36450,1.226182,1.619791,67.732685,89.475145,78.100155,0.402439,0.004563
7,7,0.3,400,60,2.452364,0.647,0.088,0.265,0.214329,1.926751,...,0.647,0.08825,0.26475,2.452364,2.977977,135.465370,164.499538,149.512899,0.402439,0.002383
8,8,0.3,400,90,3.678546,0.713,0.072,0.216,0.167542,3.062234,...,0.713,0.07175,0.21525,3.678546,4.294858,203.198055,237.242329,219.780910,0.402439,0.001621
9,9,0.3,450,30,1.226182,0.202,0.200,0.599,0.663894,0.412128,...,0.202,0.19950,0.59850,1.226182,2.040236,72.763710,121.071064,94.876506,0.402439,0.003756


In [3]:
df_rx = pd.DataFrame()
df_rx['Temp [K]'] = df['Temp [degC]'] + 273.15
df_rx['F0 [mmol/min]'] = df['Fin [mmol/min]']
df_rx['rt [min]'] = df['rt [min]']
df_rx['F [mmol/min]'] = df['Fin [mmol/min]']*(1-df['Conv'])
df_rx['W_F [g-min/mmol]'] = df['Wcat [g]']/df['Fin [mmol/min]']

df_rx

,Temp [K],F0 [mmol/min],rt [min],F [mmol/min],W_F [g-min/mmol]
0,573.15,1.226182,0.006062,1.178409,0.244662
1,573.15,2.452364,0.003049,2.386900,0.122331
2,573.15,3.678546,0.002034,3.586158,0.081554
3,623.15,1.226182,0.005382,1.086144,0.244662
4,623.15,2.452364,0.002727,2.243454,0.122331
5,623.15,3.678546,0.001834,3.433310,0.081554
6,673.15,1.226182,0.004563,0.832573,0.244662
7,673.15,2.452364,0.002383,1.926751,0.122331
8,673.15,3.678546,0.001621,3.062234,0.081554
9,723.15,1.226182,0.003756,0.412128,0.244662


In [4]:
class NH3_decomposition:
    def __init__(self, data_df, Temp):
        self.data = df_rx[df_rx['Temp [K]'] == Temp + 273.15].copy()
        self.T = Temp + 273.15
        self.P = 1 # atm

        self.data = self.data.sort_values(by='rt [min]').reset_index(drop=True)

        self.RT = self.data['rt [min]'].values
        self.F0 = self.data['F0 [mmol/min]'].values
        self.nNH3 = self.data['F [mmol/min]'].values

    def partial_pressure(self, N, N0):
        nNH3 = N
        nN2 = 0.5*(N0-N)
        nH2 = 1.5*(N0-N)
        pNH3 = self.P*nNH3/(nNH3+nN2+nH2)
        pN2 = self.P*nN2/(nNH3+nN2+nH2)
        pH2 = self.P*nH2/(nNH3+nN2+nH2)

        return np.array([pNH3, pN2, pH2])
    
    def rate_equation(self, N, k, N0):
        p = self.partial_pressure(N, N0)
        theta = 1 + k[1]*p[0] + (p[1]/k[2])**0.5 + (p[2]/k[3])**0.5
        r = k[0]*k[1]*p[0]/theta**4
        return r
    
    def ode(self, t, y, k, N0):
        N = y[0]

        rNH3 = self.rate_equation(N, k, N0)

        dNH3dt = -rNH3*0.3
        return [dNH3dt]

    def solve_ode(self, k, N0, tend):
        N0 = [N0]
        t_span = (0, tend)
        sol = solve_ivp(
            fun = lambda t, y: self.ode(t, y, k, N0),
            t_span = t_span,
            y0 = N0,
            t_eval = [tend],
            method = 'RK45'
        )
        
        return sol.y[0]
    
    def obj_fun(self, params):

        k = 10**params
        err = 0
        for i in range(len(self.RT)):
            N0 = self.F0[i]
            tend = self.RT[i]
            N_pred = self.solve_ode(k, N0, tend)

            err += np.sum((self.nNH3[i] - N_pred[0])**2)
        return err
    
    def est_params(self):

        x0 = np.ones(4)

        best_x = np.ones(4)
        best_loss = float('inf')

        for i in range(1000):
            res = minimize(
                fun = self.obj_fun,
                x0 = x0,
                method = 'L-BFGS-B',
                bounds = [(-10, 10) for _ in range(len(x0))],
                options = {'maxiter':1}
            )
            # 현재 시도의 loss 출력
            print(f'시도 {i+1} - x0: {x0}, Loss: {res.fun:.6f}, Best Loss: {best_loss:.6f}')

            if res.fun < best_loss:
                best_loss = res.fun
                best_x = res.x.copy()
                x0 = res.x.copy()
            else:
                x0 = best_x.copy()

            # x0에 랜덤 변화 추가
            x0 = x0 * (1 + np.random.randn(len(x0)) * 0.1)

        return best_x
                
        

In [5]:
a = NH3_decomposition(df_rx,450)
best_x = a.est_params()

시도 1 - x0: [1. 1. 1. 1.], Loss: 3.340907, Best Loss: inf
시도 2 - x0: [0.85927768 0.81822574 1.08176977 0.74303888], Loss: 3.340876, Best Loss: 3.340907
시도 3 - x0: [0.91576535 0.77425776 0.90459225 0.69028621], Loss: 3.340848, Best Loss: 3.340876
시도 4 - x0: [0.92178175 0.77725453 0.84265984 0.62953338], Loss: 3.340848, Best Loss: 3.340848
시도 5 - x0: [0.8199847  0.80443782 0.86059761 0.62671314], Loss: 3.340876, Best Loss: 3.340848
시도 6 - x0: [0.90303402 0.69917194 1.01839061 0.73930994], Loss: 3.340806, Best Loss: 3.340848
시도 7 - x0: [0.84900301 0.76021757 0.86248559 0.68568968], Loss: 3.340854, Best Loss: 3.340806
시도 8 - x0: [0.87463529 0.73245091 1.1253538  0.8112272 ], Loss: 3.340835, Best Loss: 3.340806
시도 9 - x0: [1.03087615 0.62306593 1.00282068 0.75211037], Loss: 3.340676, Best Loss: 3.340806
시도 10 - x0: [1.02623812 0.57402791 1.10462692 0.65546445], Loss: 3.340606, Best Loss: 3.340676
시도 11 - x0: [1.13131974 0.51495951 1.03794177 0.5491817 ], Loss: 3.340373, Best Loss: 3.340606
시

/tmp/ipykernel_18319/2968374556.py:25: RuntimeWarning: invalid value encountered in sqrt
  theta = 1 + k[1]*p[0] + (p[1]/k[2])**0.5 + (p[2]/k[3])**0.5


시도 82 - x0: [ 4.53979976 -0.93667879  1.55766419  0.88720521], Loss: 0.082549, Best Loss: 0.060711
시도 83 - x0: [ 3.96436241 -0.99049509  1.34220079  0.68135708], Loss: 0.107831, Best Loss: 0.060711
시도 84 - x0: [ 5.23236968 -0.77775825  1.27709355  0.84133267], Loss: 0.379088, Best Loss: 0.060711
시도 85 - x0: [ 4.34301546 -0.78360849  1.35489093  0.7031689 ], Loss: 0.132528, Best Loss: 0.060711
시도 86 - x0: [ 4.4118692  -1.02066862  1.35929051  0.84983051], Loss: 0.106062, Best Loss: 0.060711
시도 87 - x0: [ 5.50908789 -0.88787747  1.30299846  0.7446737 ], Loss: 1.415948, Best Loss: 0.060711
시도 88 - x0: [ 5.23324651 -0.95201626  1.29901967  0.92263239], Loss: 0.077278, Best Loss: 0.060711
시도 89 - x0: [ 4.6433309  -0.99826375  1.19662098  0.81945807], Loss: 0.065717, Best Loss: 0.060711
시도 90 - x0: [ 4.74619786 -0.98004298  1.20658251  0.7067874 ], Loss: 0.062307, Best Loss: 0.060711
시도 91 - x0: [ 5.57845996 -0.86218508  1.47698657  0.85256223], Loss: 2.109622, Best Loss: 0.060711
시도 92 - x0

In [6]:
from curses import reset_prog_mode


test_df = df[df['Temp [degC]'] == 450]
test_df
k_best = 10**best_x
def ode2(t,N):
    k = k_best

    pNH3 = N[0]/sum(N)
    pN2 = N[1]/sum(N)
    pH2 = N[2]/sum(N)
    
    theta = 1 + k[1]*pNH3 + (pN2/k[2])**0.5 + (pH2/k[3])**0.5
    r = k[0]*k[1]*pNH3/theta**4

    dNH3 = -r*0.3
    dN2 = 0.5*r*0.3
    dH2 = 1.5*r*0.3

    return [dNH3,dN2,dH2]

idx = 2
N0 = np.array([test_df.iloc[idx]['Fin [mmol/min]'],0,0])
res = solve_ivp(ode2,[0,test_df.iloc[idx]['rt [min]']],N0,method='RK45')

res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  4.282e-09 ...  7.866e-04  1.409e-03]
        y: [[ 3.679e+00  3.678e+00 ...  2.713e+00  2.468e+00]
            [ 0.000e+00  2.641e-04 ...  4.829e-01  6.055e-01]
            [ 0.000e+00  7.924e-04 ...  1.449e+00  1.816e+00]]
      sol: None
 t_events: None
 y_events: None
     nfev: 86
     njev: 0
      nlu: 0

In [7]:
df[['molF_NH3','molF_N2','molF_H2','rt [min]']]

,molF_NH3,molF_N2,molF_H2,rt [min]
0,1.178409,0.023887,0.071660,0.006062
1,2.386900,0.032732,0.098195,0.003049
2,3.586158,0.046194,0.138582,0.002034
3,1.086144,0.070019,0.210056,0.005382
4,2.243454,0.104455,0.313365,0.002727
5,3.433310,0.122618,0.367855,0.001834
6,0.832573,0.196805,0.590414,0.004563
7,1.926751,0.262806,0.788419,0.002383
8,3.062234,0.308156,0.924468,0.001621
9,0.412128,0.407027,1.221081,0.003756


In [16]:
(N0[0]-res.y[0][-1])/N0[0]

np.float64(0.329203509222975)

In [19]:
nNH3 = res.y[0][-1]
nN2 = res.y[1][-1]
nH2 = res.y[2][-1]

n_total = nNH3 + nN2 + nH2

pNH3 = nNH3/n_total
pN2 = nN2/n_total
pH2 = nH2/n_total

print(pNH3, pN2, pH2)

0.5046604873689798 0.12383487815775503 0.3715046344732651


In [26]:
X = (N0[0] - nNH3)/N0[0]
y_NH3 = (1-X)/(1+X)
y_N2 = 0.5*X/(1+X)
y_H2 = 1.5*X/(1+X)

y_NH3, y_N2, y_H2

(np.float64(0.50466048736898),
 np.float64(0.123834878157755),
 np.float64(0.371504634473265))

array([0.32920351,       -inf,       -inf])